### Radiative-convective equilibrium as an idealized problem for climate

### Background on konrad 

`konrad` is a modern implementation of a single-column radiative-convective equilibrium model. In other sections of the course you'll be using very small models for which it will be straightforward enough to read the entire implementation. `konrad` is more involved so the focus will be on framing and answering questions and interpreting the results. 

The model's technical documention -- lists of all the components and functions -- is available on [ReadTheDocs](https://konrad.readthedocs.io). The konrad authors' ["How to konrad"](https://atmtools.github.io/konrad) Jupyter book is a tutorial in using the model. All the questions we asked below are found in the tutorial but you should arrive at your own answers. 

### The assignment

In the next two labs we will use konrad to explore questions of climate, including characterizing forcing, feedbacks, and climate sensitivty. Much of the material is covered in  Kluft et al. 2019, Re-examining the first climate models: climate sensitivity of a modern radiative–convective equilibrium model, doi:[10.1175/JCLI-D-18-0774.1](https://doi.org/10.1175/JCLI-D-18-0774.1).

#### Preliminaries (do in class) 

The konrad developers have provided ...

In [ ]:
conda install -y colorcet seaborn 

In [ ]:
# Import needed Python modules 
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

import numpy as np

import konrad

In [ ]:
# Choose pressure levels according to Equation 1 in Kluft et al. 2019 
# https://konrad.readthedocs.io/_autosummary/konrad.utils.get_quadratic_pgrid.html#konrad.utils.get_quadratic_pgrid
phlev = konrad.utils.get_quadratic_pgrid(1000e2, 10, 128) 

# Initial temperature profiles and composition 
atmosphere = konrad.atmosphere.Atmosphere(phlev)
atmosphere.atmosphere_variables

konrad defines an RCE calculation as a [set of choices](https://konrad.readthedocs.io/_autosummary/konrad.core.RCE.__init__.html). Many of the most important choices (radiation, ozone, humidity, surface, cloud, convection/vertical mixing, lapse rate, and large-scale upwelling) are specified as functions rather than discrete values. Start with the default configuation: 

In [ ]:
initial_conds = atmosphere.copy()
baseline = konrad.RCE(atmosphere, timestep='24h', max_duration='150d')

Many of the sub-models have variables associated with them: 

In [ ]:
baseline.radiation.data_vars 

Two interleaved grids are used in the vertical: state variables (temperature, concentration, humidity) are on full levels (`plev`), while fluxes (especially radiation variables) are on half-levels (`phlev`). The surface is at `phlev[0]` and there's one more half-level than full level. 

Variables are undefined until the model is run, normally to convergence, which takes about 10 seconds on the LEAPangeo hub

In [ ]:
baseline.run() 

Variables will now have valid values. The first axis is time but, by default, only the values at equilibrium are available: 

In [ ]:
baseline.radiation["lw_flxu"].shape

Note that the `atmosphere` variable itself also has the new values 

In [ ]:
print(np.max(atmosphere['T'] - baseline.atmosphere['T']))

We can, for example, plot the arbitrary initial and final equilibrium profiles of temperature. We'll plot these as a function of pressure, which varies over three orders of magnitude (so we use a log scale) and decreases as altitide increases (so we'll plot an inverted y-axis). 

In [ ]:
sns.set_context("paper")

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=[12, 6], sharey=True)
ax0.semilogy(initial_conds['T'][0,:], initial_conds['plev'])
ax0.semilogy(atmosphere   ['T'][0,:], initial_conds['plev'])
ax0.set_ylabel("$p$ / Pa")
ax0.set_xlabel("$T$ / K")
ax0.invert_yaxis()

ax1.semilogy(initial_conds['H2O'][0,:], initial_conds['plev'])
ax1.semilogy(atmosphere   ['H2O'][0,:], initial_conds['plev'])
ax1.set_xlabel("water vapor / kg/kg")

sns.despine() 

What does the energy budget look like in the baseline RCE state? We can plot the four components of the radiation budget: up- and down-going radiation for solar (shortwave or sw) and terrestrial (longwave or lw) radiation. The radiation model also tells us the net heating rate. 

In [ ]:
# If we were doing this for publication we'd assign each variable a distinct color 

sns.set_context("paper")

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=[12, 6], sharey=True)
for v in ['lw_flxd', 'lw_flxu', 'sw_flxd', 'sw_flxu']:
    ax0.semilogy(baseline.radiation[v][0,:], initial_conds['phlev'], label=v)
ax0.set_ylabel("$p$ / hPa")
ax0.set_xlabel("$F$ / Wm$^{-2}$")
ax0.legend(frameon = False) 
ax0.invert_yaxis()


ax1.semilogy(baseline.radiation['net_htngrt'][0,:],       initial_conds['plev'])
ax1.set_ylabel("$p$ / hPa")
ax1.set_xlabel("$\mathcal{H}$ / K d${-1}$")

sns.despine() 

Compare the down-going and up-going amounts of solar and terrestrial radiation. Do you understand why they have the relative values they do?  

At pressures greater than 1000 Pa, or 100 hPa, i.e. in the lowest 90% of mass in the atmopheres, there's a non-zero radiative cooling rate of roughly 1 $K/d$. That's the cooling that's making the atmosphere unstable and leading to convection. 

The upper part of the atmosphere -- the stratosphere and above -- is in radiative equilibrium. 

What is the total top-of-atmosphere energy imbalance? 

#### Question 1 
What is the temperature struture of the (tropical) atmosphere under present-day conditions (that's the figure above), and how does this depend on humidity (and maybe insolation)? (A bit on how lapse rate varies with surface T?)

In [ ]:
# Show how to set the solar constant
# Show how to change the humidity - what sets the default/initial profile
# Show how to change the surface temperature 

#### Question 2
What is the instantaneous radiative forcing due to doubling CO2 concentrations from pre-industrial values, and how does this depend on humidity and temperature? 

In [ ]:
# Show how to compute radiation on a given set of atmospheric conditions, e.g. the example from How to Konrad: 

olr_ref = baseline.radiation["lw_flxu"][0,:]
# Calculate OLR at perturbed atmospheric state.
atmosphere["CO2"][:] *= 2  # double the CO2 concentration
baseline.radiation.update_heatingrates(atmosphere)

inst_rad_forcing = -(baseline.radiation["lw_flxu"][0,:] - olr_ref)

#### Question 3 
What is the effective radiative forcing due to doubling CO2 concentrations from pre-industrial values, and what is the relationship between stratospheric cooling and the radiative adjustment? 

In [ ]:
# This requires stashing the atmospheric conditions, running the model to a new equilibrium. 
# Maybe ask people to show profiles of forcing and temperture change? 

#### Question 4
What is the climate sensitivity in RCE. Does this depend on temperature? Humidity? Estimate by changing surface T; estimate again by changing atmospheric composition and running to equilibrium. 